In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
!pip install -qU trl transformers datasets bitsandbytes peft wandb accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 66.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 33.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import gc
import torch

import transformers
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, BitsAndBytesConfig
from datasets import load_dataset
from peft import LoraConfig, PeftModel, get_peft_model, prepare_model_for_kbit_training
from trl import DPOTrainer, DPOConfig, SFTConfig, SFTTrainer
import bitsandbytes as bnb

from tqdm import tqdm
import numpy as np
import warnings

warnings.filterwarnings("ignore")

In [21]:
from google.colab import userdata
from huggingface_hub import login
import wandb
hf_token = userdata.get('HF_TOKEN')
login(userdata.get('HF_TOKEN'))
wandb.login(key=userdata.get('WANDB_API_KEY'))

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [5]:
model_name = f"meta-llama/Llama-3.2-1B-Instruct"
cache_dir = "./cache"

In [6]:
# Model to fine-tune

# model_name = "thainq107/Llama-3.2-1B-Instruct-sft" f"meta-llama/Llama-3.2-1B-Instruct"
# cache_dir = "./cache"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map={"" : torch.cuda.current_device()},
    token=userdata.get('HF_TOKEN'),
    cache_dir=cache_dir,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)
base_model.config.use_cache = False
base_model

config.json:   0%|          | 0.00/877 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (v_proj): Linear4bit(in_features=2048, out_features=512, bias=False)
          (o_proj): Linear4bit(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (up_proj): Linear4bit(in_features=2048, out_features=8192, bias=False)
          (down_proj): Linear4bit(in_features=8192, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((2048,), eps=1e-05)
      )
    )
    (norm): LlamaRMSNorm((2048,), 

In [7]:
tokenizer = AutoTokenizer.from_pretrained(
    model_name, cache_dir=cache_dir, trust_remote_code=True
)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id

tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [8]:
dataset = load_dataset("thainq107/Vi-Alpaca-Preference")
dataset, dataset["train"][0]

README.md:   0%|          | 0.00/506 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/61.9M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/1.95M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/65017 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

(DatasetDict({
     train: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 65017
     })
     test: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 2000
     })
 }),
 {'id': 'alpaca-7294',
  'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
  'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
  'rejected': 'Câu này không có lỗi ngữ pháp.'})

In [9]:
# Lấy subset cho từng split
dataset["train"] = dataset["train"].select(range(200))
dataset["test"] = dataset["test"].select(range(100))

# In ra đúng format như ban đầu
dataset, dataset["train"][0]

(DatasetDict({
     train: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 200
     })
     test: Dataset({
         features: ['id', 'question', 'chosen', 'rejected'],
         num_rows: 100
     })
 }),
 {'id': 'alpaca-7294',
  'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
  'chosen': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
  'rejected': 'Câu này không có lỗi ngữ pháp.'})

In [10]:
# QLoRA configuration
peft_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj"
    ]
)

## **SFT**

In [11]:
def formatting_prompt_with_chat_template(example):
    conversation = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": example["question"]},
        {"role": "assistant", "content": example["chosen"]},
    ]
    prompt = tokenizer.apply_chat_template(
        conversation, tokenize=False, add_generation_prompt=False
    )
    return prompt

In [12]:
model_name

'meta-llama/Llama-3.2-1B-Instruct'

In [13]:
hyperparameters = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 2,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "bf16": True,
}
MAX_LENGTH = 216

In [14]:
# Use wandb
import wandb
wandb.init(
    project="vi-alpaca-preference",
    name="llama-3.2-1b-4bit-sft"
)


In [15]:
!nvidia-smi

Thu Aug 14 03:41:57 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   44C    P0             27W /   70W |    1332MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [31]:
# import torch
# torch.cuda.empty_cache()

In [16]:
SFT_OUTPUT_DIR = f"Llama-3.2-1B-Instruct-sft"

sft_config = SFTConfig(
    **{ **hyperparameters, "output_dir":
       SFT_OUTPUT_DIR , "max_length": MAX_LENGTH}
)
sft_trainer = SFTTrainer(
    model=base_model,
    peft_config=peft_config,
    processing_class=tokenizer,
    args=sft_config,
    train_dataset=dataset['train'],
    formatting_func=formatting_prompt_with_chat_template
)

sft_trainer.train()

Applying formatting function to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Adding EOS to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=50, training_loss=2.98488037109375, metrics={'train_runtime': 317.2463, 'train_samples_per_second': 1.261, 'train_steps_per_second': 0.158, 'total_flos': 461109408940032.0, 'train_loss': 2.98488037109375})

In [22]:
HF_USER = "dauvannam321"
MODEL_NAME = "Llama-3.2-1B-Instruct-200"
sft_trainer.save_model(SFT_OUTPUT_DIR)
sft_trainer.push_to_hub(f"{HF_USER}/{MODEL_NAME}-SFT", token=hf_token)

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

events.out.tfevents.1755142936.e8c9b5581dfc.297.0:   0%|          | 0.00/6.61k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/5.75k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dauvannam321/Llama-3.2-1B-Instruct-sft/commit/fa632ee94666019b4e7b97ab4a566cf0fdee7c15', commit_message='dauvannam321/Llama-3.2-1B-Instruct-200-SFT', commit_description='', oid='fa632ee94666019b4e7b97ab4a566cf0fdee7c15', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dauvannam321/Llama-3.2-1B-Instruct-sft', endpoint='https://huggingface.co', repo_type='model', repo_id='dauvannam321/Llama-3.2-1B-Instruct-sft'), pr_revision=None, pr_num=None)

## **DPO**

In [24]:
# Model to fine-tune

model_name = "dauvannam321/Llama-3.2-1B-Instruct-SFT"
cache_dir = "./cache"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    trust_remote_code=True,
    device_map={"" : torch.cuda.current_device()},
    token=userdata.get('HF_TOKEN'),
    cache_dir=cache_dir,
    torch_dtype=torch.bfloat16,
    quantization_config=bnb_config,
)
base_model.config.use_cache = False
base_model

adapter_config.json:   0%|          | 0.00/944 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 2048)
    (layers): ModuleList(
      (0-15): 16 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=2048, bias=False)
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2048, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=2048, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): lora.Linear4bit(
            (base_layer): Linear4bit(in_features=2048, out_features=512, bias=False)
            (lora_dropout): ModuleDict(
       

In [25]:
def convert_to_conversational_preference_format(example):
    return {
        "id": example["id"],
        "prompt": [{"role": "system",
                    "content": "You are a helpful assistant."},
                   {"role": "user",
                    "content": example["question"]}],
        "chosen": [{"role": "assistant",
                    "content": example["chosen"]}],
        "rejected": [{"role": "assistant",
                      "content": example["rejected"]}],
    }

dpo_dataset = dataset.map(convert_to_conversational_preference_format)


Map:   0%|          | 0/200 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [28]:
dpo_dataset['train'][0]

{'id': 'alpaca-7294',
 'question': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
 'chosen': [{'content': 'Không có lỗi ngữ pháp. Câu này đã chính xác.',
   'role': 'assistant'}],
 'rejected': [{'content': 'Câu này không có lỗi ngữ pháp.',
   'role': 'assistant'}],
 'prompt': [{'content': 'You are a helpful assistant.', 'role': 'system'},
  {'content': 'Xác định và sửa lỗi ngữ pháp.\n\nTôi đã đi đến cửa hàng.',
   'role': 'user'}]}

In [29]:
hyperparameters = {
    "per_device_train_batch_size": 2,
    "gradient_accumulation_steps": 4,
    "gradient_checkpointing": True,
    "learning_rate": 3e-5,
    "logging_steps": 200,
    "num_train_epochs": 2,
    "save_strategy": "no",
    "overwrite_output_dir": True,
    "optim": "paged_adamw_8bit",
    "warmup_steps": 200,
    "bf16": True,
}
MAX_LENGTH = 216

In [30]:
# Use wandb
import wandb
wandb.init(
    project="vi-alpaca-preference",
    name="llama-3.2-1b-4bit-dpo"
)


train/epoch,▁
train/global_step,▁
train/mean_token_accuracy,▁
train/num_tokens,▁
total_flos,461109408940032.0
train/epoch,2
train/global_step,50
train/mean_token_accuracy,0.51598
train/num_tokens,64402
train_loss,2.98488
train_runtime,317.2463


In [31]:
DPO_OUTPUT_DIR = f"Llama-3.2-1B-Instruct-dpo"
dpo_args = DPOConfig(
    **{ **hyperparameters, "output_dir":
       DPO_OUTPUT_DIR, "max_length": MAX_LENGTH }
)

dpo_trainer = DPOTrainer(
    base_model,
    args=dpo_args,
    train_dataset=dpo_dataset['train'],
    processing_class=tokenizer,
    peft_config=peft_config,
)
dpo_trainer.train()


Extracting prompt in train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Applying chat template to train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/200 [00:00<?, ? examples/s]

Step,Training Loss


TrainOutput(global_step=50, training_loss=0.6741983795166016, metrics={'train_runtime': 820.3397, 'train_samples_per_second': 0.488, 'train_steps_per_second': 0.061, 'total_flos': 0.0, 'train_loss': 0.6741983795166016, 'epoch': 2.0})

In [32]:
dpo_trainer.save_model(DPO_OUTPUT_DIR)
dpo_trainer.push_to_hub(f"{HF_USER}/{MODEL_NAME}-DPO-full", token=hf_token)

events.out.tfevents.1755143681.e8c9b5581dfc.297.1:   0%|          | 0.00/7.04k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/45.1M [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.33k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dauvannam321/Llama-3.2-1B-Instruct-dpo/commit/8ab6958efa7ce734237fac8f1230134498794b73', commit_message='dauvannam321/Llama-3.2-1B-Instruct-200-DPO-full', commit_description='', oid='8ab6958efa7ce734237fac8f1230134498794b73', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dauvannam321/Llama-3.2-1B-Instruct-dpo', endpoint='https://huggingface.co', repo_type='model', repo_id='dauvannam321/Llama-3.2-1B-Instruct-dpo'), pr_revision=None, pr_num=None)